In [0]:
# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input, Embedding
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.utils.np_utils import to_categorical
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

# NLTK
nltk.download("stopwords")
nltk.download("punkt")
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

# Other
import re
import io
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# load data from drive
from google.colab import drive
drive.mount('/content/drive')
df_restaurants = pd.read_csv('/content/drive/My Drive/yelp_review_english.csv')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = df_restaurants[['stars','text']]
data['sentiment']=['pos' if (x>3) else 'neg' for x in data['stars']]

In [0]:
def clean_text(text):
    
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    stops.remove('not')
    text = [w for w in text if not w in stops and len(w) >= 3]

    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [0]:
# clean review
data['clean_review'] = data['text'].apply(clean_text)

In [0]:
vocabulary_size = 10000 
max_length = 100
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(data['text'])

sequences = tokenizer.texts_to_sequences(data['text'])
X = pad_sequences(sequences, maxlen=100) 

In [0]:
Y = pd.get_dummies(data['stars']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 40)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2480307, 100) (2480307, 5)
(620077, 100) (620077, 5)


In [0]:
#CNN + LSTM
from keras.layers import Bidirectional
import keras
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=100)) #50, 50
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu')) #100,3
    model_conv.add(MaxPooling1D(pool_size=4)) # 2
    model_conv.add(Dropout(0.2))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(5, activation='softmax'))
    #model_conv.add(Dense(2, activation='softmax'))
    #optim = keras.optimizers.Adam(lr=0.005)
    model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    #model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

model_conv = create_conv_model()
print(model_conv.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 64)            32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 505       
Total para

In [0]:
model_conv.fit(X_train, Y_train,
          validation_data=(X_test, Y_test),
          batch_size=128,
          epochs=10,
          verbose=1)

Train on 2480307 samples, validate on 620077 samples
Epoch 1/10
2480307/2480307 [==============================] - 1738s 701us/step - loss: 0.7947 - categorical_accuracy: 0.6498 - val_loss: 0.7447 - val_categorical_accuracy: 0.6724
Epoch 2/10
2480307/2480307 [==============================] - 1734s 699us/step - loss: 0.7453 - categorical_accuracy: 0.6718 - val_loss: 0.7336 - val_categorical_accuracy: 0.6766
Epoch 3/10
2003456/2480307 [=======================>......] - ETA: 4:59 - loss: 0.7309 - categorical_accuracy: 0.6781Buffered data was truncated after reaching the output size limit.